# Symbolic regression of a dynamical system

In this example, Kozax is applied to recover the state equations of the Lotka-Volterra system. The candidate solutions are integrated as a system of differential equations, after which the predictions are compared to the true observations to determine a fitness score.

In [1]:
# Specify the cores to use for XLA
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=10'

import jax
import diffrax
import jax.numpy as jnp
import jax.random as jr
import diffrax

from kozax.genetic_programming import GeneticProgramming
from kozax.fitness_functions.ODE_fitness_function import ODEFitnessFunction
from kozax.environments.SR_environments.lotka_volterra import LotkaVolterra

These device(s) are detected:  [CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7), CpuDevice(id=8), CpuDevice(id=9)]


First the data is generated, consisting of initial conditions, time points and the true observations. Kozax provides the Lotka-Volterra environment, which is integrated with Diffrax.

In [2]:
def get_data(key, env, dt, T, batch_size=20):
    x0s = env.sample_init_states(batch_size, key)
    ts = jnp.arange(0, T, dt)

    def solve(env, ts, x0):
        solver = diffrax.Dopri5()
        dt0 = 0.001
        saveat = diffrax.SaveAt(ts=ts)

        system = diffrax.ODETerm(env.drift)

        # Solve the system given an initial conditions
        sol = diffrax.diffeqsolve(system, solver, ts[0], ts[-1], dt0, x0, saveat=saveat, max_steps=500, 
                                  adjoint=diffrax.DirectAdjoint(), stepsize_controller=diffrax.PIDController(atol=1e-7, rtol=1e-7, dtmin=0.001))
        
        return sol.ys

    ys = jax.vmap(solve, in_axes=[None, None, 0])(env, ts, x0s) #Parallelize over the batch dimension
    
    return x0s, ts, ys

key = jr.PRNGKey(0)
data_key, gp_key = jr.split(key)

T = 30
dt = 0.2
env = LotkaVolterra()

# Simulate the data
data = get_data(data_key, env, dt, T, batch_size=4)
x0s, ts, ys = data

For the fitness function, we used the ODEFitnessFunction that uses Diffrax to integrate candidate solutions. It is possible to select the solver, time step, number of steps and a stepsize controller to balance efficiency and accuracy. To ensure convergence of the genetic programming algorithm, constant optimization is applied to the best candidates at every generation. The constant optimization is performed with a couple of simple evolutionary steps that adjust the values of the constants in a candidate. The hyperparameters that define the constant optimization are `constant_optimization_N_offspring` (number of candidates with different constants should be sampled for each candidate), `constant_optimization_steps` (number of iterations of constant optimization for each candidate), `optimize_constants_elite` (number of candidates that constant optimization is applied to), `constant_step_size_init` (initial value of the step size for sampling constants) and `constant_step_size_decay` (the rate of decrease of the step size over generations).

In [3]:
#Define the nodes and hyperparameters
operator_list = [
        ("+", lambda x, y: jnp.add(x, y), 2, 0.5), 
        ("-", lambda x, y: jnp.subtract(x, y), 2, 0.1), 
        ("*", lambda x, y: jnp.multiply(x, y), 2, 0.5), 
    ]

variable_list = [["x" + str(i) for i in range(env.n_var)]]
layer_sizes = jnp.array([env.n_var])

population_size = 100
num_populations = 10
num_generations = 50

#Initialize the fitness function and the genetic programming strategy
fitness_function = ODEFitnessFunction(solver=diffrax.Dopri5(), dt0 = 0.01, stepsize_controller=diffrax.PIDController(atol=1e-6, rtol=1e-6, dtmin=0.001), max_steps=300)

strategy = GeneticProgramming(num_generations, population_size, fitness_function, operator_list, variable_list, layer_sizes, num_populations = num_populations,
                        size_parsimony=0.003, constant_optimization_method="evolution", constant_optimization_N_offspring = 50, constant_optimization_steps = 3, 
                        optimize_constants_elite=100, constant_step_size_init=0.1, constant_step_size_decay=0.99)

Input data should be formatted as: ['x0', 'x1'].


Kozax provides a fit function that receives the data and a random key. However, it is also possible to run Kozax with an easy loop consisting of evaluating and evolving. This is useful as different input data can be provided during evaluation. In symbolic regression of dynamical systems, it helps to first optimize on a small part of the time points, and provide the full data trajectories only after a couple of generations.

In [4]:
# Sample the initial population
population = strategy.initialize_population(gp_key)

# Define the number of timepoints to include in the data
end_ts = int(ts.shape[0]/2)

for g in range(num_generations):
    if g == 25: # After 25 generations, use the full data
        end_ts = ts.shape[0]

    key, eval_key, sample_key = jr.split(key, 3)
    # Evaluate the population on the data, and return the fitness
    fitness, population = strategy.evaluate_population(population, (x0s, ts[:end_ts], ys[:,:end_ts]), eval_key)

    # Print the best solution in the population in this generation
    best_fitness, best_solution = strategy.get_statistics(g)
    print(f"In generation {g+1}, best fitness = {best_fitness:.4f}, best solution = {strategy.expression_to_string(best_solution)}")

    # Evolve the population until the last generation. The fitness should be given to the evolve function.
    if g < (num_generations-1):
        population = strategy.evolve_population(population, fitness, sample_key)

In generation 1, best fitness = 1.3755, best solution = [1.29 - 2.42*x0, 0.0769 - 0.336*x1]
In generation 2, best fitness = 1.3496, best solution = [-0.893*x0*x1 + 1.1, 1.6*x0 - 2.06]
In generation 3, best fitness = 1.3289, best solution = [-0.331*x0*x1, x0 - 0.413*x1 + 0.0743]
In generation 4, best fitness = 1.3097, best solution = [x1*(-0.35*x0 - 0.246) + 1.23, 0.364 - 0.36*x1]
In generation 5, best fitness = 1.2857, best solution = [-1.6*x0 - 0.248*x1 + 1.4, x0 - 0.415*x1 - 0.35]
In generation 6, best fitness = 1.2720, best solution = [-1.72*x0 - 0.245*x1 + 1.42, 0.822*x0 - 0.384*x1 - 0.265]
In generation 7, best fitness = 1.2277, best solution = [-1.16*x0*x1 + x0 + 0.551, 0.107*x0 - 0.228*x1 - 0.16]
In generation 8, best fitness = 1.2014, best solution = [-0.639*x0*x1 + x0 + 0.461, 0.146*x0 - 0.197*x1 - 0.208]
In generation 9, best fitness = 0.9929, best solution = [-2.6*x0*(-0.0276*x0 + 0.273*x1) + x0 + 0.461, 0.146*x0 - 0.197*x1 - 0.208]
In generation 10, best fitness = 0.8489, b

In [5]:
strategy.print_pareto_front()

Complexity: 2, fitness: 2.8892037868499756, equations: [-0.776, -0.763]
Complexity: 4, fitness: 1.9611964225769043, equations: [-2.14*x0, -0.682]
Complexity: 6, fitness: 1.3917429447174072, equations: [-1.98*x0, -0.285*x1]
Complexity: 8, fitness: 1.3050819635391235, equations: [-2.23*x0, x0 - 0.41*x1]
Complexity: 10, fitness: 1.2563836574554443, equations: [0.835 - 2.31*x0, x0 - 0.537*x1]
Complexity: 12, fitness: 1.2215551137924194, equations: [1.11 - 2.31*x0, x0 - 0.45*x1 - 0.276]
Complexity: 14, fitness: 0.8544017672538757, equations: [-0.845*x0*(x1 - 2.48), 0.141*x0 - 0.271*x1]
Complexity: 16, fitness: 0.06887245923280716, equations: [-0.415*x0*(x1 - 2.7), 0.104*x0*x1 - 0.404*x1]
Complexity: 18, fitness: 0.06406070291996002, equations: [-0.398*x0*(x1 - 2.67), 0.103*x0*x1 - 0.415*x1 - 0.000306]
Complexity: 20, fitness: 0.016078392043709755, equations: [-0.402*x0*(x1 - 2.76), 0.0993*x0*x1 - 0.397*x1]


Instead of using evolution to optimize the constants, Kozax also offers gradient-based optimization. For gradient optimization, it is possible to specify the optimizer, the number of candidates to apply constant optimization to, the initial learning rate and the learning rate decay over generation. These two methods are provided as either can be more effective or efficient for different problems.

In [6]:
import optax

strategy = GeneticProgramming(num_generations, population_size, fitness_function, operator_list, variable_list, layer_sizes, num_populations = num_populations,
                        size_parsimony=0.003, constant_optimization_method="gradient", constant_optimization_steps = 15, optimizer_class = optax.adam,
                        optimize_constants_elite=100, constant_step_size_init=0.025, constant_step_size_decay=0.95)

Input data should be formatted as: ['x0', 'x1'].


In [7]:
key = jr.PRNGKey(0)
data_key, gp_key = jr.split(key)

T = 30
dt = 0.2
env = LotkaVolterra()

# Simulate the data
data = get_data(data_key, env, dt, T, batch_size=4)
x0s, ts, ys = data

# Sample the initial population
population = strategy.initialize_population(gp_key)

# Define the number of timepoints to include in the data
end_ts = int(ts.shape[0]/2)

for g in range(num_generations):
    if g == 25: # After 25 generations, use the full data
        end_ts = ts.shape[0]

    key, eval_key, sample_key = jr.split(key, 3)
    # Evaluate the population on the data, and return the fitness
    fitness, population = strategy.evaluate_population(population, (x0s, ts[:end_ts], ys[:,:end_ts]), eval_key)

    # Print the best solution in the population in this generation
    best_fitness, best_solution = strategy.get_statistics(g)
    print(f"In generation {g+1}, best fitness = {best_fitness:.4f}, best solution = {strategy.expression_to_string(best_solution)}")

    # Evolve the population until the last generation. The fitness should be given to the evolve function.
    if g < (num_generations-1):
        population = strategy.evolve_population(population, fitness, sample_key)

In generation 1, best fitness = 1.4242, best solution = [1.41 - 2.33*x0, -0.226*x1 - 0.145]
In generation 2, best fitness = 1.3698, best solution = [1.16 - 2.29*x0, 0.113 - 0.317*x1]
In generation 3, best fitness = 1.2569, best solution = [-0.49*x0*x1 + 0.674, 0.907*x0 - 0.261*x1 - 0.681]
In generation 4, best fitness = 1.1925, best solution = [-0.283*x0*x1 + 0.391, 0.882*x0 - 0.272*x1 - 0.805]
In generation 5, best fitness = 1.1737, best solution = [-0.276*x0*x1 + 0.392, 0.729*x0 - 0.286*x1 - 0.678]
In generation 6, best fitness = 1.1737, best solution = [-0.276*x0*x1 + 0.392, 0.729*x0 - 0.286*x1 - 0.678]
In generation 7, best fitness = 1.1177, best solution = [(1.41 - 0.532*x1)*(x0 - 0.218), x0 - 0.427*x1]
In generation 8, best fitness = 1.0575, best solution = [2*x0*(0.495 - 0.22*x1), (-1.18*x0 + x1)*(0.00345*x0 - 0.399)]
In generation 9, best fitness = 0.9400, best solution = [(0.686 - 0.268*x1)*(2*x0 + 0.161), (-1.26*x0 + x1)*(0.0518*x0 - 0.375)]
In generation 10, best fitness = 0

In [8]:
strategy.print_pareto_front()

Complexity: 2, fitness: 2.889601707458496, equations: [-0.789, -0.746]
Complexity: 4, fitness: 2.231682300567627, equations: [1.05 - x0, -0.336]
Complexity: 6, fitness: 1.392594814300537, equations: [-1.95*x0, -0.291*x1]
Complexity: 8, fitness: 1.3738036155700684, equations: [-1.62*x0, x0 - 0.407*x1]
Complexity: 10, fitness: 1.3001673221588135, equations: [-0.276*x0*x1, x0 - 0.386*x1]
Complexity: 12, fitness: 1.226672649383545, equations: [-0.274*x0*x1 + 0.218, x0 - 0.407*x1]
Complexity: 14, fitness: 0.916778028011322, equations: [x0*(1.33 - 0.49*x1), 0.311*x0 - 0.327*x1]
Complexity: 16, fitness: 0.04582058638334274, equations: [2*x0*(0.537 - 0.202*x1), x1*(0.103*x0 - 0.415)]
Complexity: 18, fitness: 0.04011291265487671, equations: [1.86*x0*(0.574 - 0.215*x1), x1*(0.103*x0 - 0.415)]
